# TSP

In [2]:
import pandas as pd

dist = pd.read_excel('simple-nlp.xlsx', sheet_name='tsp', index_col=0)
dist

,C1,C2,C3,C4,C5,C6
C1,0,16,63,21,20,66
C2,57,0,40,46,69,42
C3,23,11,0,55,53,47
C4,71,53,58,0,47,5
C5,27,79,53,35,0,30
C6,57,47,51,17,24,0


In [36]:
def convert_2d_array_to_dict(arr):
    '''Converts a 2-d ndarray, DataFrame or Series to a parameter dictionary.'''
    if isinstance(arr, pd.DataFrame) or isinstance(arr, pd.Series):
        arr = arr.values
    return {(i+1,j+1): arr[i,j] for i in range(arr.shape[0]) for j in range(arr.shape[1])}

dist_dict = convert_2d_array_to_dict(dist)

In [54]:
m.city.display()

city : Dim=0, Dimen=1, Size=6, Domain=Integers, Ordered=True, Bounds=(1, 6)
    Virtual


In [103]:
import pyomo.environ as pe
import numpy as np

m = pe.ConcreteModel()

m.city = pe.RangeSet(1, dist.shape[0])
m.x = pe.Var(m.city, domain=pe.NonNegativeIntegers, 
             initialize={i: i for i in list(range(1, dist.shape[0]+1))})

m.dist = pe.Param(m.city, m.city, initialize=dist_dict)

big_m = 1000
def obj(model):
    result = 0.
    for i in m.city:
        if i == 6:
            result += m.dist[pe.value(m.x[i]), pe.value(m.x[1])] + \
                       big_m * np.abs(m.x[i] - m.x[1])
        else:
            result += m.dist[pe.value(m.x[i]), pe.value(m.x[i+1])] + \
                       big_m * np.abs(m.x[i] - m.x[1])
    return result
m.obj = pe.Objective(rule=obj)

In [106]:
# This is how you solve!
opt = pe.SolverFactory('ipopt')
result = opt.solve(m, tee=True)

Ipopt 3.12.12: 
==> Warning: Treating 0 binary and 6 integer variables as continous.


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.12, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       11

Total number of variables............................:        6
                     variables with only lower bounds:        6
                variables with lower and upper bounds:       

 107  5.4527840e+02 0.00e+00 1.02e+02  -1.0 1.20e+11    -  4.66e-08 1.09e-19f 28
 108  5.4527839e+02 0.00e+00 3.00e+02  -1.0 3.74e+09    -  4.66e-10 1.84e-17f 38
 109  5.4527838e+02 0.00e+00 1.02e+02  -1.0 3.73e+11    -  1.52e-08 1.39e-19f 26
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 110  5.4527838e+02 0.00e+00 1.02e+02  -1.0 3.81e+09    -  4.57e-10 2.32e-18f 41
 111  5.4527838e+02 0.00e+00 1.02e+02  -1.0 3.81e+11    -  1.46e-08 1.70e-20f 29
 112  5.4527836e+02 0.00e+00 3.00e+02  -1.0 1.19e+10    -  1.46e-10 2.30e-18f 41
 113  5.4527836e+02 0.00e+00 1.02e+02  -1.0 1.19e+12    -  4.78e-09 5.45e-21f 29
 114  5.4527836e+02 0.00e+00 1.02e+02  -1.0 1.21e+10    -  1.43e-10 2.88e-19f 44
 115  9.2552243e+03 0.00e+00 5.00e+02  -1.0 1.21e+12    -  4.59e-09 1.44e-12w  1
 116  7.5099952e+03 0.00e+00 3.00e+02  -1.0 3.79e+10    -  9.10e-10 4.60e-11w  1
 117  1.5508367e+07 0.00e+00 5.00e+02  -1.0 1.23e+09    -  1.52e-11 2.53e-06w  1
 118  5.4527836e+02 0.00e+00

 313  5.8009273e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.81e-13f 11
 314  5.7974850e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 7.02e-14f 13
 315  5.7971435e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 8.78e-15f 16
 316  5.7970797e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.19e-15f 18
 317  5.7970759e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 1.10e-15f 19
 318  5.7970459e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 5.49e-16f 20
 319  5.7970443e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 3.43e-17f 24
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 320  2.3503991e+03 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.88e-10w  1
 321  7.5643594e+03 0.00e+00 5.00e+02  -1.0 6.16e+09    -  2.87e-08 2.88e-10w  1
 322  7.5791767e+03 0.00e+00 3.00e+02  -1.0 5.82e+07    -  5.75e-09 3.04e-10w  1
 323  5.7970441e+02 0.00e+00 1.03e+02  -1.0 1.98e+08    -  2.87e-08 4.29e-18f 26
 324  5.7970441e+02 0.00e+00

 462  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
 463  2.3504043e+03 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.88e-10w  1
 464  7.5643697e+03 0.00e+00 5.00e+02  -1.0 6.16e+09    -  2.87e-08 2.88e-10w  1
 465  7.5791925e+03 0.00e+00 3.00e+02  -1.0 5.82e+07    -  5.75e-09 3.04e-10w  1
 466  5.7970440e+02 0.00e+00 1.03e+02  -1.0 1.98e+08    -  2.87e-08 2.04e-24f 47
 467  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
 468  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
 469  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 470  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
 471  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
 472  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
 473  5.7970440e+02 0.00e+00

 610  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
 611  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
 612  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
 613  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
 614  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
 615  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
 616  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
 617  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
 618  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
 619  2.3504083e+03 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.88e-10w  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 620  7.5643778e+03 0.00e+00 5.00e+02  -1.0 6.16e+09    -  2.87e-08 2.88e-10w  1
 621  7.5792048e+03 0.00e+00

...

2480  7.5792493e+03 0.00e+00 3.00e+02  -1.0 5.82e+07    -  5.75e-09 3.04e-10w  1
2481  5.7970440e+02 0.00e+00 1.03e+02  -1.0 1.98e+08    -  2.87e-08 2.04e-24f 47
2482  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
2483  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
2484  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
2485  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
2486  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
2487  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
2488  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
2489  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2490  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
2491  2.3504230e+03 0.00e+00

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2620  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
2621  2.3504233e+03 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.88e-10w  1
2622  7.5644077e+03 0.00e+00 5.00e+02  -1.0 6.16e+09    -  2.87e-08 2.88e-10w  1
2623  7.5792505e+03 0.00e+00 3.00e+02  -1.0 5.82e+07    -  5.75e-09 3.04e-10w  1
2624  5.7970440e+02 0.00e+00 1.03e+02  -1.0 1.98e+08    -  2.87e-08 2.04e-24f 47
2625  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
2626  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
2627  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
2628  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
2629  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2630  5.7970440e+02 0.00e+00

2766  7.5792515e+03 0.00e+00 3.00e+02  -1.0 5.82e+07    -  5.75e-09 3.04e-10w  1
2767  5.7970440e+02 0.00e+00 1.03e+02  -1.0 1.98e+08    -  2.87e-08 2.04e-24f 47
2768  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
2769  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2770  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
2771  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
2772  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
2773  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
2774  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
2775  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
2776  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
2777  2.3504237e+03 0.00e+00

2913  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
2914  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
2915  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
2916  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
2917  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
2918  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
2919  5.7970440e+02 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.04e-24f 48
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2920  2.3504240e+03 0.00e+00 1.03e+02  -1.0 6.10e+09    -  2.87e-08 2.88e-10w  1
2921  7.5644091e+03 0.00e+00 5.00e+02  -1.0 6.16e+09    -  2.87e-08 2.88e-10w  1
2922  7.5792526e+03 0.00e+00 3.00e+02  -1.0 5.82e+07    -  5.75e-09 3.04e-10w  1
2923  5.7970440e+02 0.00e+00 1.03e+02  -1.0 1.98e+08    -  2.87e-08 2.04e-24f 47
2924  5.7970440e+02 0.00e+00

WARNING: Loading a SolverResults object with a warning status into
    model=unknown;
        message from solver=Ipopt 3.12.12\x3a Maximum Number of Iterations
        Exceeded.


In [107]:
m.obj.expr()

7579.253099598283